In [2]:
import pyro
import pyro.distributions as dist
from pyro.infer.mcmc import MCMC, HMC, NUTS
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import torch.distributions.constraints as constraints

import numpy as np
import scipy.stats as stats
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

%matplotlib inline

In [3]:
assert pyro.__version__.startswith('1.8.1')
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run the notebook cells on CI.
#smoke_test = 'CI' in os.environ

In [4]:
nn_decoder = nn.Sequential(nn.Linear(20, 100), nn.Softplus(), nn.Linear(100, 784), nn.Sigmoid())

In [5]:
device = torch.device('cpu')
# if torch.cuda.is_available():
#     device = torch.device('cuda:0')

In [608]:
from torch.distributions.utils import (_sum_rightmost, broadcast_all,
                                       lazy_property, tril_matrix_to_vec,
                                       vec_to_tril_matrix)

In [621]:
x = torch.tensor(np.random.sample(size=15))
y = torch.tensor(np.random.sample(size=15))
xs = torch.cat((x, y))

In [622]:
xs.shape

torch.Size([30])

In [612]:
x = torch.tensor(np.random.sample(size=15))
r = vec_to_tril_matrix(x)
r

tensor([[0.4136, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6976, 0.8194, 0.0000, 0.0000, 0.0000],
        [0.7773, 0.3456, 0.8523, 0.0000, 0.0000],
        [0.4432, 0.1132, 0.9493, 0.9556, 0.0000],
        [0.4181, 0.3966, 0.4868, 0.8207, 0.4597]], dtype=torch.float64)

In [613]:
x

tensor([0.4136, 0.6976, 0.8194, 0.7773, 0.3456, 0.8523, 0.4432, 0.1132, 0.9493,
        0.9556, 0.4181, 0.3966, 0.4868, 0.8207, 0.4597], dtype=torch.float64)

In [660]:
torch.eye(5)*0.1

tensor([[0.1000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1000]])

In [655]:
def make_pd_mat(size, tensor_of_vectors):
    x = vec_to_tril_matrix(tensor_of_vectors)
    for idx in range(size):
        x[:, idx, idx] = torch.exp(x[:, idx, idx])
    z_scale = torch.bmm(x, x.transpose(1, 2))
    return z_scale

In [754]:
import torch.nn.functional as F
# define the PyTorch module that parameterizes the
# diagonal gaussian distribution q(z|x)
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        super(Encoder, self).__init__()
        self.z_dim = z_dim
        self.input_dim = input_dim
        # setup the three linear transformations used
        self.fc1 = nn.Linear(input_dim[0]*input_dim[1], hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        val = int((z_dim**2 + z_dim) / 2)
#         print(val)
        self.fc22 = nn.Linear(hidden_dim, val) # z_dim**2)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x.reshape(x.shape[0], self.input_dim[0] * self.input_dim[1])))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        A_vect = self.fc22(hidden)
#         A = A_vect.reshape((A_vect.shape[0], self.z_dim, self.z_dim))
#         A = torch.tril(A)
        A = vec_to_tril_matrix(A_vect)
        z_scale = torch.bmm(A, A.transpose(1, 2))
#         z_scale = make_pd_mat(self.z_dim, A_vect)
        z_scale.add_(torch.eye(self.z_dim)*1e-4)
        return z_loc, z_scale


# define the PyTorch module that parameterizes the
# observation likelihood p(x|z)
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        super(Decoder, self).__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, input_dim)
        self.fc22 = nn.Linear(hidden_dim, input_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        mu = self.fc21(hidden)
        sigma = torch.exp(self.fc22(hidden))
        return mu, sigma


# define a PyTorch module for the VAE
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, input_dim,
        z_dim=5, hidden_dim=250, use_cuda=False):
        super(VAE, self).__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim, input_dim=input_dim)
        # self.decoder = Decoder(z_dim, hidden_dim, input_dim=input_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, sigma, x):
        # register PyTorch module `decoder` with Pyro
        # pyro.module("decoder", self.decoder)
        
        for i in pyro.plate("batch_loop", x.shape[0]):
            
            # setup hyperparameters for prior p(z)
            mu_loc = torch.zeros(x.shape[0], self.z_dim, dtype=x.dtype, device=x.device)
            mu_scale = torch.eye(x[0].shape[1]).reshape(
                (1,x[0].shape[1],x[0].shape[1])).repeat(x.shape[0], 1, 1)
            
            # sample from prior (value will be sampled by guide when computing the ELBO
            mu = pyro.sample("latent_{}".format(i), dist.MultivariateNormal(mu_loc, mu_scale).to_event(1))
            scale = pyro.param("scale_{}".format(i), torch.eye(x[0].shape[1]).reshape(
                (1,x[0].shape[1],x[0].shape[1])).repeat(x.shape[0], 1, 1))
            
            for j in pyro.plate("data_loop_{}".format(i), x.shape[1]):
                pyro.sample("obs_{}_{}".format(j, i), dist.MultivariateNormal(mu[i, :], scale[i, :]),
                                obs=x[i, j, :])          
            
    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, sigma, x):
        
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        for i in pyro.plate("batch_loop", x.shape[0]):
            
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder.forward(x)

            # sample the latent code z
            z = pyro.sample("latent_{}".format(i), dist.MultivariateNormal(z_loc, z_scale).to_event(1))
            
            scale = pyro.param("scale_{}".format(i), torch.eye(x[0].shape[1]).reshape(
                (1,x[0].shape[1],x[0].shape[1])).repeat(x.shape[0], 1, 1))

In [771]:
X = []
means = []
sigma_mats = []
sigma_0s = []
for idx in range(500):
    random_means = np.random.sample(size=5) 
    
    data, mu_vector, sigma_mat, sigma_mat_0 = multivar_random(5, [1]*5, [0.5,.5,.5,.5], num_samples=500)
    X.append(data)
    means.append(mu_vector)
    sigma_mats.append(sigma_mat)
    sigma_0s.append(sigma_mat_0)

In [ ]:
# clear param store
pyro.clear_param_store()

no_instances = 20000
input_dim = (500, 5)
mu = stats.norm.rvs(size=input_dim)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim, z_dim=5)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())
train_loader = DataLoader(X, batch_size=500, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

train_elbo = []

for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    print("Epoch: ", epoch, end = "\r")
    for x in train_loader:
        # x = x.cuda()
        epoch_loss += svi.step(torch.eye(input_dim[1]), x)

    # report training diagnostics
    if not epoch % 2:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))

# NUM_EPOCHS = 1 if smoke_test else 100
# TEST_FREQUENCY = 5    
        
# train_elbo = []
# test_elbo = []
# # training loop
# for epoch in range(NUM_EPOCHS):
#     total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
#     train_elbo.append(-total_epoch_loss_train)
#     print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

#     if epoch % TEST_FREQUENCY == 0:
#         # report test diagnostics
#         total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
#         test_elbo.append(-total_epoch_loss_test)
#         print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))

[epoch 000]  average training loss: 10228.6071
[epoch 002]  average training loss: 13888.4650


In [ ]:
plt.figure(figsize=(15, 9))
plt.plot([i for i in range(len(train_elbo))], train_elbo)
plt.xlabel("Epoch")
plt.ylabel("ELBO loss")

In [759]:
count = 0
for name, value in pyro.get_param_store().items():
    print(name)

encoder$$$fc1.weight
encoder$$$fc1.bias
encoder$$$fc21.weight
encoder$$$fc21.bias
encoder$$$fc22.weight
encoder$$$fc22.bias
scale_0
scale_1
scale_2
scale_3
scale_4
scale_5
scale_6
scale_7
scale_8
scale_9


In [760]:
vae.encoder.eval()

Encoder(
  (fc1): Linear(in_features=2500, out_features=250, bias=True)
  (fc21): Linear(in_features=250, out_features=5, bias=True)
  (fc22): Linear(in_features=250, out_features=15, bias=True)
  (softplus): Softplus(beta=1, threshold=20)
)

In [762]:
X[0].shape
vae.encoder.forward(X[0].unsqueeze(0))

(tensor([[ 0.5175,  0.5493, -0.1156,  0.7151,  0.2803]],
        grad_fn=<AddmmBackward0>),
 tensor([[[ 0.0005, -0.0003, -0.0002,  0.0019,  0.0019],
          [-0.0003,  0.0342,  0.0021,  0.0104,  0.0060],
          [-0.0002,  0.0021,  0.0004, -0.0018, -0.0020],
          [ 0.0019,  0.0104, -0.0018,  0.0604,  0.0285],
          [ 0.0019,  0.0060, -0.0020,  0.0285,  0.0593]]],
        grad_fn=<AddBackward0>))

In [763]:
means[0]

tensor([ 0.6586,  0.6002, -0.6715,  1.0164,  0.6153], dtype=torch.float64)

In [765]:
sigma_0s[0]

tensor([[0.1027, 0.0814, 0.1489, 0.0659, 0.1382],
        [0.0814, 0.1473, 0.2139, 0.1397, 0.2136],
        [0.1489, 0.2139, 0.5736, 0.0636, 0.2113],
        [0.0659, 0.1397, 0.0636, 0.2158, 0.2671],
        [0.1382, 0.2136, 0.2113, 0.2671, 0.3754]], dtype=torch.float64)

In [768]:
# random_means = np.random.sample(size=5) 
# x_i, y = multivar_random(5, random_means, [1,1,1,1], num_samples=500)[:2]
x_i, mu_vector, sigma, sigma_0 = multivar_random(5, [0] * 5, [.5]*4, 500)

vae.encoder.forward(x_i.reshape(1, 500, 5))

(tensor([[ 0.3845, -0.2769, -0.1771,  0.5380,  0.2525]],
        grad_fn=<AddmmBackward0>),
 tensor([[[ 0.0198,  0.0007, -0.0150,  0.0210,  0.0430],
          [ 0.0007,  0.1966,  0.1053,  0.0098,  0.1318],
          [-0.0150,  0.1053,  0.3383,  0.3370,  0.1371],
          [ 0.0210,  0.0098,  0.3370,  0.6337,  0.1409],
          [ 0.0430,  0.1318,  0.1371,  0.1409,  0.2276]]],
        grad_fn=<AddBackward0>))

In [769]:
post_mean = get_posterior_mean_mv_gaussian(50, sigma, sigma_0, mu_vector, x_i)
post_mean

tensor([ 0.1331, -0.3002,  0.0038,  0.3147, -0.0914], dtype=torch.float64)

In [770]:
post_sigma = get_posterior_cov_mv_gaussian(50, sigma, sigma_0)
post_sigma

tensor([[ 7.9513e-03, -8.7204e-05, -1.4412e-04, -1.3693e-03,  2.0596e-03],
        [-8.7204e-05,  5.5606e-03,  8.4741e-05,  4.8326e-04,  7.2660e-04],
        [-1.4412e-04,  8.4741e-05,  3.4021e-05,  2.2387e-04,  6.7872e-06],
        [-1.3693e-03,  4.8326e-04,  2.2387e-04,  2.0614e-03,  1.0721e-04],
        [ 2.0596e-03,  7.2660e-04,  6.7872e-06,  1.0721e-04,  1.0570e-03]],
       dtype=torch.float64)

In [689]:
# write a function to find this exact posterior 

def get_posterior_cov_mv_gaussian(n, sigma, sigma_0):
    sum_part = sigma_0 + 1/n * sigma
    sum_part = np.linalg.inv(sum_part)
    p1 = np.matmul(sigma_0, sum_part)
    p2 = np.matmul(p1, 1/n * sigma)
    return(p2)

def get_posterior_mean_mv_gaussian(n, sigma, sigma_0, mu_0, data):
    sum_part = sigma_0 + 1/n * sigma
    sum_part = np.linalg.inv(sum_part)
    
    middle_part = data.mean(axis=0)
    
    p1 = np.matmul(sigma_0, sum_part)
    p2 = np.matmul(p1, middle_part)
    
    p3 = np.matmul(1/n * sigma, sum_part)
    p4 = np.matmul(p3, mu_0)
    return(p2 + p4)

In [8]:
def multivar_random(size, normal_params, gamma_params, num_samples):   

    # simulate two random covariance matrices (PSD) for sigma and sigma_0
    mat = np.random.gamma(gamma_params[0], gamma_params[1], size=size**2).reshape((size, size))
    sigma_mat = torch.tensor(np.dot(mat, mat.transpose()))
    
    mat_0 = np.random.gamma(gamma_params[2], gamma_params[3], size=size**2).reshape((size, size))
    sigma_mat_0 = torch.tensor(np.dot(mat_0, mat_0.transpose()))
    
    # simulate some random means
    mu_vector = torch.tensor(np.random.multivariate_normal(normal_params, sigma_mat_0, size=1))[0]

    # now simulate multivariate gaussian
    data = torch.tensor(np.random.multivariate_normal(mu_vector, sigma_mat, size=num_samples).astype("float32"))
    
    return(data, mu_vector, sigma_mat, sigma_mat_0)

In [9]:
train_loader

NameError: name 'train_loader' is not defined

In [ ]:
# clear param store
pyro.clear_param_store()

no_instances = 20000
input_dim = 2
mu = stats.norm.rvs(size=input_dim)

# Generate a positive definite matrix
sigma = stats.norm.rvs(size=(input_dim, input_dim))
sigma[np.triu_indices(input_dim)] = 0
sigma += np.diag(np.abs(stats.norm.rvs(size=input_dim)))
sigma = np.matmul(sigma.transpose(), sigma) # inverse cholesky decomposition

dataset = stats.multivariate_normal.rvs(mu, sigma, size=no_instances)
dataset = torch.as_tensor(dataset, dtype=torch.float32)
dataset = TensorDataset(dataset)
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim, z_dim=2)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, in train_loader:
        #x = x.cuda()
        epoch_loss += svi.step(x)

    # report training diagnostics
    if not epoch % 10:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))

In [64]:
test = stats.multivariate_normal.rvs(mu, sigma, size=no_instances)
test = torch.as_tensor(test, dtype=torch.float32)
vae.encoder(test)[0]

NameError: name 'sigma' is not defined

In [ ]:
no_instances

In [ ]:
# Generating new instances (replications) from the trained VAE
new_instances = vae.new_instances(100000)

print("True means")
print(mu)
print("Empirical means of replications:")
print(new_instances.mean(0))

print("----------------------------------------")

print("True covariance matrix")
print(sigma)
print("Empirical covariance matrix of replications:")
print(np.cov(new_instances, rowvar=False))

In [ ]:
# clear param store
pyro.clear_param_store()
loc = 5
scale = 20
data = torch.tensor(np.random.normal(loc=loc, scale=scale, size=1000))
dataset = TensorDataset(data)
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, in train_loader:
        #x = x.cuda()
        epoch_loss += svi.step(x)

    # report training diagnostics
    if not epoch % 10:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))